In [ ]:
import vk_api.vk_api
import random
import requests
import bs4
import time


from vk_api.bot_longpoll import VkBotLongPoll
from vk_api.bot_longpoll import VkBotEventType

### Bot realization:

In [ ]:
class Bot:

    def __init__(self, api_token, group_id, server_name: str="Empty"):

        self.server_name = server_name
        self.vk = vk_api.VkApi(token=api_token)
        self.long_poll = VkBotLongPoll(self.vk, group_id)
        self.vk_api = self.vk.get_api()
        self.smth_is_active = False
        self.top_capitals = dict()
        self.top_gallows = dict()
        self.balance = dict()
        self.member_list = dict()


    def start(self):
        """ Основная фоновая работа """
        for event in self.long_poll.listen():

            if event.type == VkBotEventType.MESSAGE_NEW and event.object['text']:

                if (event.object['text'].lower() == 'миксер топ столицы'
                    and self.smth_is_active == False):
                    self.show_top(event.object.peer_id, self.top_capitals)

                elif (event.object['text'].lower() == 'миксер топ виселица'
                    and self.smth_is_active == False):
                    self.show_top(event.object.peer_id, self.top_gallows)

                elif (event.object['text'].lower() == 'миксер столицы'
                    and self.smth_is_active == False):
                    self.smth_is_active = True
                    self.game_capitals(event.object.peer_id)

                elif (event.object['text'].lower() == 'миксер меню'
                    and self.smth_is_active == False):
                    self.menu(event.object.peer_id)

                elif (event.object['text'].lower() == 'миксер баланс'
                    and self.smth_is_active == False):
                    self.show_balance(event.object.peer_id, event.object.from_id)

                elif (event.object['text'][:13].lower() == 'миксер рандом'
                    and self.smth_is_active == False):
                    self.randomizer(event.object.peer_id, event.object['text'][13:])

                elif (event.object['text'][:10].lower() == 'миксер кто'
                    and self.smth_is_active == False):
                    self.who(event.object.peer_id, event.object['text'][10:])

                elif (event.object['text'].lower() == 'миксер топ аниме'
                    and self.smth_is_active == False):
                    self.get_top_anime(event.object.peer_id)

                elif (event.object['text'].lower() == 'миксер 21' or\
                event.object['text'].lower() == 'миксер очко'\
                    and self.smth_is_active == False):
                        self.cmth_is_active = True
                        self.game_21(event.object.peer_id, event.object.from_id)


    def get_user_name(self, user_id):
        """ Получаем имя пользователя"""
        return self.vk_api.users.get(user_id=user_id)[0]['first_name']

    def get_user_lastname(self, user_id):
        """ Получаем фамилию пользователя"""
        return self.vk_api.users.get(user_id=user_id)[0]['last_name']

    def get_user_fullname(self, user_id):
        """ Получаем полное имя пользователя """
        return (self.vk_api.users.get(user_id=user_id)[0]['first_name']+ ' '
                + self.vk_api.users.get(user_id=user_id)[0]['last_name'])

    def get_user_city(self, user_id):
        """ Получаем город пользователя"""
        return self.vk_api.users.get(user_id=user_id,
                                     fields="city")[0]["city"]['title']

    def send_message(self, peer_id, message):
        """ Посылает сообщение """
        self.vk_api.messages.send(peer_id=peer_id,
                                  message=message,
                                  random_id = random.randint(100000, 999999))

    def show_balance(self, peer_id, from_id):
        """ Показывает баланс """
        try:
            msg = (self.get_user_name(from_id) + ', у тебя на счету ' +
               str(self.balance[self.get_user_fullname(from_id)]) + ' монет')
        except KeyError:
            msg = (self.get_user_name(from_id) +
                   ', у тебя на счету 0 монет...\n\nНу ты и бомж...')
        self.send_message(peer_id, msg)

    def show_top(self, peer_id, data):
        if len(self.top_capitals.items()) == 0:
            msg = 'Ни у кого пока нет баллов за эту игру'
            self.send_message(peer_id, msg)
        else:
            list_top = list(self.data.items())
            list_top.sort(key=lambda i: -i[1])
            msg = ''
            for el in list_top:
                msg += str(el[0]) + ' : ' + str(el[1]) + '\n'
            self.send_message(peer_id, msg)

    def randomizer(self, peer_id, string):
        random.seed()
        string = string.split()
        borders = list()
        for el in string:
            if el.isdigit():
                borders.append(int(el))
        try:
            self.send_message(peer_id, random.choice(thinking_phrase) +
                                                  str(random.randint(borders[0], borders[1])))
        except:
            self.send_message(peer_id, 'Неправильный формат ввода')

    def menu(self, peer_id):
        """ Запуск меню """
        menu_msg = ('1) Игры\n2) Баланс\n3) Рандомайзер\n4) Список лучших аниме\n5) Помощь' +
            '\n6) Инстаграм создателя\n\nОтправьте цифру пункта, который хотите выбрать')
        self.send_message(peer_id, menu_msg)
        random.seed()
        for event in self.long_poll.listen():
            if event.type == VkBotEventType.MESSAGE_NEW and event.object['text']:
                if event.object['text'][:1] == '1':
                    msg = '1) Столицы\n2) Виселица\n3) 21(Очко)\n0) Выход из меню'
                    self.send_message(peer_id, msg)
                    for event2 in self.long_poll.listen():
                        if event2.type == VkBotEventType.MESSAGE_NEW and event2.object['text']:
                            if event2.object['text'][:1] == '1':
                                self.smth_is_active = True
                                self.game_capitals(event2.object.peer_id)
                                return
                            if event2.object['text'][:1] == '2':
                                self.smth_is_active = True
                                self.game_gallows(event2.object.peer_id)
                                return
                            if event2.object['text'][:1] == '3':
                                self.smth_is_active = True
                                self.game_21(event2.object.peer_id, event2.object.from_id)
                                return
                            if event2.object['text'][:1] == '0':
                                msg = 'Окей, вышел'
                                self.send_message(peer_id, msg)
                                return

                if event.object['text'][:1] == '2':
                    self.show_balance(peer_id, event.object.from_id)
                    return

                if event.object['text'][:1] == '3':
                    self.send_message(peer_id,
                                      'Введите границы рандома в формате "х у" без кавычек')
                    for event in self.long_poll.listen():
                        if event.type == VkBotEventType.MESSAGE_NEW and event.object['text']:
                            try:
                                borders = event.object['text']
                                borders = list(map(int, borders.split()))
                                self.send_message(peer_id, random.choice(thinking_phrase) +
                                                  str(random.randint(borders[0], borders[1])))
                            except:
                                self.send_message(peer_id, 'Неправильный формат ввода')
                            return

                if event.object['text'][:1] == '4':
                    self.get_top_anime(peer_id)
                    return

                if event.object['text'][:1] == '5':
                    pass

                if event.object['text'][:1] == '6':
                    msg = 'Чекай чекай:\n https://www.instagram.com/mikpetrovsky/'
                    self.send_message(peer_id, msg)
                    return

    def chiphr(self, string):
        res = ''
        for el in string:
            if el != '-':
                res += '_ '
            else:
                res += '- '
        return res[:-1]


    def game_capitals(self, peer_id):
        """ Игра в столицы """
        rounds = 3
        random.seed()
        tmp_top = dict()
        i = random.randint(0, len(country) - 1)
        msg = 'Назовите столицу ' + country[i][0] + '\n\n' + self.chiphr(country[i][1])
        self.send_message(peer_id, msg)
        for event in self.long_poll.listen():
            if event.type == VkBotEventType.MESSAGE_NEW and event.object['text']:
                if event.object['text'].lower() == country[i][1]:
                    self.send_message(event.object.peer_id,
                                      self.get_user_name(event.object.from_id)
                                      + right_choice_phrase[random.randint(0,4)])
                    rounds -= 1
                    dict_ind = self.get_user_fullname(event.object.from_id)
                    self.top_capitals[dict_ind] = self.top_capitals.get(dict_ind, 0) + 1
                    tmp_top[dict_ind] = tmp_top.get(dict_ind, 0) + 1
                    self.balance[dict_ind] = self.balance.get(dict_ind, 0) + 1
                    if rounds == 0:
                        self.smth_is_active = False
                        list_top = list(tmp_top.items())
                        list_top.sort(key=lambda i: -i[1])
                        msg = 'Ну все, хватит \n\n Рейтинг за эту игру:\n'
                        for el in list_top:
                            msg += str(el[0]) + ' : ' + str(el[1]) + '\n'
                        self.send_message(event.object.peer_id, msg)
                        return
                    i = random.randint(0, 4)
                    msg = 'Назовите столицу ' + country[i][0] + '\n\n' + self.chiphr(country[i][1])
                    self.send_message(event.object.peer_id, msg)
                else:
                    self.send_message(event.object.peer_id,
                                      self.get_user_name(event.object.from_id)
                                      + wrong_choice_phrase[random.randint(0,4)])

    def game_gallows(self, peer_id):
        """ Игра в виселицу """
        self.smth_is_active = False
        return

    def game_21(self, peer_id, from_id):
        """ Игра в 21(очко) """
        msg = 'Предварительные правила:\nОтправляй "еще", если хочешь получить еще карту' + \
        ', или "хватит" если уже достаточно.\nПодберись как можно ближе к 21, но не перескочи.'
        self.send_message(peer_id, msg)
        time.sleep(3)
        random.seed()
        cards_for_game = list(cards)
        card1 = random.choice(cards_for_game)
        cards_for_game.remove(card1)
        card2 = random.choice(cards_for_game)
        cards_for_game.remove(card2)
        hand = [card1, card2]
        pl_points = cards_points[card1] + cards_points[card2]
        msg = self.get_user_name(from_id) + ', твои первые две карты:\n' + card1 + ' и ' + card2\
        + ' (' + str(pl_points) + ' очк.)'
        self.send_message(peer_id, msg)
        for event in self.long_poll.listen():
            if (event.type == VkBotEventType.MESSAGE_NEW and event.object['text']
            and event.object.from_id == from_id):
                if event.object['text'].strip().lower() == 'стоп игра':
                    self.send_message(peer_id, 'Игра прервана')
                    self.smth_is_active = False
                    return
                if event.object['text'].strip().lower() == 'еще'\
                or event.object['text'].strip().lower() == 'ещё':
                    ad_card = random.choice(cards_for_game)
                    pl_points += cards_points[ad_card]
                    cards_for_game.remove(ad_card)
                    hand.append(ad_card)
                    time.sleep(1)
                    msg = self.get_user_name(from_id) + ', ты вытянул:\n' + ad_card + \
                    '\n\nТеперь у тебя в руке:\n' + ', '.join(hand) + ' (' \
                    + str(pl_points) + ' очк.)'
                    self.send_message(peer_id, msg)
                    if pl_points > 21:
                        time.sleep(0.5)
                        msg = 'У тебя больше 21 - ты проиграл :('
                        self.send_message(peer_id, msg)
                        self.smth_is_active = False
                        return
                if event.object['text'].strip().lower() == 'хватит':
                    msg = 'У тебя ' + str(pl_points) + ' очк.\nТеперь мой ход'
                    self.send_message(peer_id, msg)
                    time.sleep(1)
                    my_card1 = random.choice(cards_for_game)
                    cards_for_game.remove(my_card1)
                    my_card2 = random.choice(cards_for_game)
                    cards_for_game.remove(my_card2)
                    my_hand = [my_card1, my_card2]
                    my_points = cards_points[my_card1] + cards_points[my_card2]
                    msg = 'Мои первые две карты это:\n' + my_card1 + ' и ' +\
                    my_card2 + ' (' + str(my_points) + ' очк.)'
                    self.send_message(peer_id, msg)
                    time.sleep(2)
                    while my_points < 18:
                        msg = 'Хочу еще, и вытягиваю...\n'
                        my_ad_card = random.choice(cards_for_game)
                        cards_for_game.remove(my_ad_card)
                        my_hand.append(my_ad_card)
                        my_points += cards_points[my_ad_card]
                        msg += my_ad_card + ' (' + str(my_points) + ' очк.)'
                        self.send_message(peer_id, msg)
                        time.sleep(1)
                        msg = '\n\nТеперь у меня в руке:\n' + ', '.join(hand) + ' (' \
                        + str(my_points) + ' очк.)'
                        self.send_message(peer_id, msg)
                        time.sleep(2)
                    msg = 'Мне хватит, у меня ' + str(my_points) + ' очк.'
                    self.send_message(peer_id, msg)
                    if my_points > 21:
                        msg = 'У меня больше 21, в этот раз ты выйграл.'
                        self.send_message(peer_id, msg)
                        self.smth_is_active = False
                        return
                    elif my_points > pl_points:
                        msg = 'Я победил, не расстраивайся, заходи еще'
                        self.send_message(peer_id, msg)
                    elif my_points < pl_points:
                        msg = 'Ладно, ты победил, забирай выйгрыш'
                        self.send_message(peer_id, msg)
                    else:
                        msg = 'Ничья! Ставка возвращена назад'
                        self.send_message(peer_id, msg)


        self.smth_is_active = False
        return

    def get_top_anime(self, peer_id, count=5):
        msg = ('1) All Anime\n2) Top Airing\n3) Top Upcoming\n4) Top TV Series\n' +
        '5) Top Movies\n6) Top OVAs\n7) Top Specials\n' +
        '8) Most Popular\n9) Most Favorited')
        self.send_message(peer_id, msg)
        for event2 in self.long_poll.listen():
            if event2.type == VkBotEventType.MESSAGE_NEW and event2.object['text']:
                html = requests.get("https://myanimelist.net/topanime.php?type=" +
                                    anime_cat[int(event2.object['text'][:1]) - 1])
                soup = bs4.BeautifulSoup(html.text, "html.parser")
                an_list = {}
                for anime in soup.select(".ranking-list", limit=count):
                    url = anime.select(".hoverinfo_trigger")[0]['href']
                    anime = anime.select(".hoverinfo_trigger")[0].findAll("img")[0]
                    name = anime['alt'].split(":")[1].strip(" ")
                    an_list[name] = url
                msg = ''
                for el in an_list:
                    msg += el + ':\n' + an_list[el] + '\n\n'
                self.send_message(peer_id, msg)
                return


    def who(self, peer_id, string):
        random.seed()
        string.strip()
        if string == 'бот':
            self.send_message(peer_id, 'Ты совсем? Ты не можешь понять, кто в беседе бот' +\
            ', а кто человек? Досвидания')
            return
        member_tmp = self.vk_api.messages.getConversationMembers(peer_id=peer_id)['profiles']
        for el in member_tmp:
            self.member_list[el['id']] = el['first_name'] + ' ' + el['last_name']
        winner = str(random.choice(list(self.member_list.keys())))
        msg = (random.choice(sure_phrase) + string +
        ' @id' + winner + ' (' + self.member_list[int(winner)] + ')')
        self.send_message(peer_id, msg)

### Starting bot work:

In [ ]:
bot1 = Bot('api_token here', 183896777, "server1")
bot1.start()

### Some data arrays:

In [ ]:
country = (('Абхазии', 'сухум'), ('Австралии', 'канберра'),
           ('Австрии', 'вена'), ('Азербайджана', 'баку'),
           ('Албании', 'тирана'), ('Алжира', 'алжир'),
           ('Анголы', 'луанда'), ('Андорры', 'андорра-ла-велья'),
           ('Аргентины', 'буэнос-айрес'), ('Армении', 'ереван'),
           ('Афганистана', 'кабул'), ('Бангладеша', 'дакка'),
           ('Барбадоса', 'бриджтаун'), ('Бахрейна', 'манама'),
           ('Белоруси', 'минск'), ('Белиза', 'бельмопан'),
           ('Бельгии', 'брюссель'), ('Болгарии', 'софия'),
           ('Боснии и Герцеговины', 'сараево'), ('Бразилии', 'бразилиа'),
           ('Ватикана', 'ватикан'), ('Великобритании', 'лондон'),
           ('Венгрии', 'будапешт'), ('Венесуэлы', 'каракас'),
           ('Вьетнама', 'ханой'), ('Гайаны', 'джорджтаун'),
           ('Гаити', 'порт-о-пренс'), ('Ганы', 'аккра'),
           ('Гватемалы', 'гватемала'), ('Германии', 'берлин'),
           ('Гондураса', 'тегусигальпа'), ('Греции', 'афины'),
           ('Грузии', 'тбилиси'), ('Дании', 'копенгаген'),
           ('Доминиканы', 'санто-доминго'), ('Египта', 'каир'),
           ('Израиля', 'иерусалим'), ('Индии', 'нью-дели'),
           ('Индонезии', 'джакарта'), ('Иордании', 'амман'),
           ('Ирака', 'багдад'), ('Ирана', 'тегеран'),
           ('Ирландии', 'дублин'), ('Исландии', 'рейкьявик'),
           ('Испании', 'мадриж'), ('Италии', 'рим'),
           ('Казахстана', 'нур-султан'), ('Камбоджи', 'пномпень'),
           ('Канады', 'оттава'), ('Катара', 'доха'),
           ('Кении', 'найроби'), ('Кипра', 'никосия'),
           ('Киргизии', 'бишкек'), ('Китая', 'пекин'),
           ('Коста-Рики', 'сан-хосе'), ('Кубы', 'гавана'),
           ('Латвии', 'рига'), ('Ливана', 'бейрут'),
           ('Ливии', 'триполи'), ('Литвы', 'вильнюс'),
           ('Лихтенштейна', 'вадуц'), ('Люксембурга', 'люксембург'),
           ('Мадагаскара', 'антананариву'), ('Македонии', 'скопье'),
           ('Малайзии', 'куала-лампур'), ('Мальдив', 'мале'),
           ('Мальты', 'валлетта'), ('Марокко', 'рабат'),
           ('Мексики', 'мехико'), ('Молдовы', 'кишенёв'),
           ('Монако', 'монако'), ('Непала', 'катманду'),
           ('Нигера', 'ниамей'), ('Нидерландов', 'амстердам'),
           ('Новой Зеландии', 'веллингтон'), ('Норвегии', 'осло'),
           ('ОАЭ', 'абу-даби'), ('Пакистана', 'исламабад'),
           ('Панамы', 'панама'), ('Перу', 'лима'),
           ('Польши', 'варшава'), ('Португалии', 'лиссабон'),
           ('России', 'москва'), ('Румынии', 'бухарест'),
           ('Саудовской Аравии', 'эр-рияд'), ('Северной Кореи', 'пхеньян'),
           ('Сенегала', 'дакар'), ('Сербии', 'белград'),
           ('Сингапур', 'сингапур'), ('Сирии', 'дамаск'),
           ('Словакии', 'братислава'), ('Словении', 'любляна'),
           ('США', 'вашингтон'), ('Таджикистана', 'душамбе'),
           ('Тайвани', 'тайбэй'), ('Таиланда', 'бангкок'),
           ('Туниса', 'тунис'), ('Турции', 'анкара'),
           ('Узбекистана', 'ташкент'), ('Украины', 'киев'),
           ('Уругвая', 'монтевидео'), ('Филиппин', 'манила'),
           ('Финляндии', 'хельсински'), ('Франции', 'париж'),
           ('Хорватии', 'загреб'), ('Черногории', 'подгорица'),
           ('Чехии', 'прага'), ('Чили', 'сантьяго'),
           ('Швейцарии', 'берн'), ('Швеции', 'стокгольм'),
           ('Эквадора', 'кито'), ('Эстонии', 'таллин'),
           ('Эфиопии', 'аддис-абеба'), ('Южной Кореи', 'сеул'),
           ('Ямайки', 'кингстон'), ('Японии', 'токио') )

right_choice_phrase = (', в точку', ', да, правильно', ', абсолютно верно',
                       ', и это... \n\n ПРАВИЛЬНО!', ', проверяю...\n\n Верно, поздравляю',
                       ', красава, верно', ', you are absolutely right!')

wrong_choice_phrase = (', неправильно', ', ошибочка', ', в этом твоя ошибка',
                       ', не не, не то', ', не сходится с базой данных')

thinking_phrase = ('И это будет...\n...\n', 'Ну пусть будет ',
                   'Звезды велят мне назвать ', 'Мой внутренний генератор рандома говорит ',
                   'Я выбираю ')

sure_phrase = ('Очевидно, что ', 'С вероятностью 100% ',
               'Будьте уверены, ', 'Отвечаю, ', 'Не, ну тут все понятно, ')

anime_cat = ('', 'airing', 'upcoming', 'tv',
             'movie', 'ova', 'special', 'bypopularity', 'favorite')

cards = ['шестерка'] * 4 + ['семерка'] * 4 + ['восьмерка'] * 4 + \
 ['девятка'] * 4 + ['десятка'] * 4 + ['валет'] * 4 + \
 ['дама'] * 4 + ['король'] * 4 + ['туз'] * 4

cards_points = {'шестерка' : 6, 'семерка' : 7,
                'восьмерка' : 8, 'девятка' : 9, 'десятка' : 10,
                'валет' : 2, 'дама' : 3, 'король' : 4, 'туз' : 11}

triger_phrases = {'мило' : 'КАК ЭТО МЫЫЫЫЫЫЛО', 'удобно' : 'УДОБНО', \
'красиво':'КРАСИВО', 'полезно' : 'ПОЛЕЗНО'}